In [4]:
!pip install chardet

In [1]:
import pandas as pd
import numpy as np
import chardet

In [2]:
with open("ner_dataset.csv", "rb") as f:
    encoding = chardet.detect(f.read())["encoding"]

In [3]:
encoding

'Windows-1252'

In [4]:
data = pd.read_csv("ner_dataset.csv",encoding=encoding)
data

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O
...,...,...,...,...
1048570,NaN,they,PRP,O
1048571,NaN,responded,VBD,O
1048572,NaN,to,TO,O
1048573,NaN,the,DT,O


In [5]:
from itertools import chain
def get_dict_map(data, token_or_tag):
    tok2idx = {}
    idx2tok = {}

    if token_or_tag == 'token':
        vocab = list(set(data['Word'].to_list()))
    else:
        vocab = list(set(data['Tag'].to_list()))

    idx2tok = {idx:tok for  idx, tok in enumerate(vocab)}
    tok2idx = {tok:idx for  idx, tok in enumerate(vocab)}
    return tok2idx, idx2tok
token2idx, idx2token = get_dict_map(data, 'token')
tag2idx, idx2tag = get_dict_map(data, 'tag')

In [52]:
token2idx

{'brokerage': 0,
 '131-5': 1,
 'Taiwan-grown': 2,
 'voyage': 3,
 'slit': 4,
 'Amy': 5,
 'sniper': 6,
 '10,000': 7,
 'shouting': 8,
 'fruitful': 9,
 'Jan.': 10,
 'Bhat': 11,
 'disrepute': 12,
 'labor': 13,
 'ecotourism': 14,
 'close': 15,
 'ethical': 16,
 'appointments': 17,
 'Erich': 18,
 'scorers': 19,
 'avarice': 20,
 'Jiangxi': 21,
 '259': 22,
 'one-quarter': 23,
 '99.29': 24,
 'next': 25,
 'living': 26,
 '423-1': 27,
 'Terje': 28,
 '37-degree': 29,
 '20-year-olds': 30,
 'Cubana': 31,
 'SPENDTHRIFT': 32,
 'multitudes': 33,
 'fast-growing': 34,
 'doled': 35,
 'Sometimes': 36,
 'Graphic': 37,
 'tampering': 38,
 'Muhammed': 39,
 'coercive': 40,
 'breeding': 41,
 'honked': 42,
 'sacks': 43,
 'decide': 44,
 'Bueno': 45,
 'warrant': 46,
 'inquired': 47,
 'convictions': 48,
 'preserve': 49,
 'Recep': 50,
 'Aka': 51,
 'points': 52,
 'worm': 53,
 'high-technology': 54,
 'spilled': 55,
 'collect': 56,
 'American-made': 57,
 'specialize': 58,
 'links': 59,
 'Torgelow': 60,
 'Speakers': 61,
 'S

In [61]:
data['Word_idx'] = data['Word'].map(token2idx)
data['Tag_idx'] = data['Tag'].map(tag2idx)
data_fillna = data.fillna(method='ffill', axis=0)
# Groupby and collect columns
data_group = data_fillna.groupby(
['Sentence #'],as_index=False
)['Word', 'POS', 'Tag', 'Word_idx', 'Tag_idx'].agg(lambda x: list(x))

<ipython-input-61-f8b936d5d036>:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  data_group = data_fillna.groupby(


In [62]:
data_group.head()

,Sentence #,Word,POS,Tag,Word_idx,Tag_idx
0,Sentence: 1,"[Thousands, of, demonstrators, have, marched, ...","[NNS, IN, NNS, VBP, VBN, IN, NNP, TO, VB, DT, ...","[O, O, O, O, O, O, B-geo, O, O, O, O, O, B-geo...","[15368, 152, 31608, 8311, 29672, 5092, 14411, ...","[3, 3, 3, 3, 3, 3, 10, 3, 3, 3, 3, 3, 10, 3, 3..."
1,Sentence: 10,"[Iranian, officials, say, they, expect, to, ge...","[JJ, NNS, VBP, PRP, VBP, TO, VB, NN, TO, JJ, J...","[B-gpe, O, O, O, O, O, O, O, O, O, O, O, O, O,...","[24095, 18672, 33686, 20460, 33112, 32321, 322...","[0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ..."
2,Sentence: 100,"[Helicopter, gunships, Saturday, pounded, mili...","[NN, NNS, NNP, VBD, JJ, NNS, IN, DT, NNP, JJ, ...","[O, O, B-tim, O, O, O, O, O, B-geo, O, O, O, O...","[24888, 24396, 27438, 24690, 17823, 28794, 217...","[3, 3, 15, 3, 3, 3, 3, 3, 10, 3, 3, 3, 3, 3, 1..."
3,Sentence: 1000,"[They, left, after, a, tense, hour-long, stand...","[PRP, VBD, IN, DT, NN, JJ, NN, IN, NN, NNS, .]","[O, O, O, O, O, O, O, O, O, O, O]","[15989, 7586, 22840, 27141, 31231, 15522, 5221...","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]"
4,Sentence: 10000,"[U.N., relief, coordinator, Jan, Egeland, said...","[NNP, NN, NN, NNP, NNP, VBD, NNP, ,, NNP, ,, J...","[B-geo, O, O, B-per, I-per, O, B-tim, O, B-geo...","[24614, 34663, 28272, 7699, 29765, 29796, 8016...","[10, 3, 3, 8, 6, 3, 15, 3, 10, 3, 0, 3, 0, 3, ..."


In [14]:
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

def get_pad_train_test_val(data_group, data):

    #get max token and tag length
    n_token = len(list(set(data['Word'].to_list())))
    n_tag = len(list(set(data['Tag'].to_list())))

    #Pad tokens (X var)
    tokens = data_group['Word_idx'].tolist()
    maxlen = max([len(s) for s in tokens])
    pad_tokens = pad_sequences(tokens, maxlen=maxlen, dtype='int32', padding='post', value= n_token - 1)

    #Pad Tags (y var) and convert it into one hot encoding
    tags = data_group['Tag_idx'].tolist()
    pad_tags = pad_sequences(tags, maxlen=maxlen, dtype='int32', padding='post', value= tag2idx["O"])
    n_tags = len(tag2idx)
    pad_tags = [to_categorical(i, num_classes=n_tags) for i in pad_tags]

    #Split train, test and validation set
    tokens_, test_tokens, tags_, test_tags = train_test_split(pad_tokens, pad_tags, test_size=0.1, train_size=0.9, random_state=2020)
    train_tokens, val_tokens, train_tags, val_tags = train_test_split(tokens_,tags_,test_size = 0.25,train_size =0.75, random_state=2020)

    print(
        'train_tokens length:', len(train_tokens),
        '\ntrain_tokens length:', len(train_tokens),
        '\ntest_tokens length:', len(test_tokens),
        '\ntest_tags:', len(test_tags),
        '\nval_tokens:', len(val_tokens),
        '\nval_tags:', len(val_tags),
    )

    return train_tokens, val_tokens, test_tokens, train_tags, val_tags, test_tags

train_tokens, val_tokens, test_tokens, train_tags, val_tags, test_tags = get_pad_train_test_val(data_group, data)

train_tokens length: 32372 
train_tokens length: 32372 
test_tokens length: 4796 
test_tags: 4796 
val_tokens: 10791 
val_tags: 10791


In [64]:
import numpy as np
import tensorflow
from tensorflow.keras import Sequential, Model, Input
from tensorflow.keras.layers import LSTM, Embedding, Dense, Dropout
from tensorflow.keras.utils import plot_model

In [65]:
input_dim = len(list(set(data['Word'].to_list())))+1
output_dim = 64
input_length = max([len(s) for s in data_group['Word_idx'].tolist()])
n_tags = len(tag2idx)

In [70]:
def get_lstm_model():
    model = Sequential()

    # Add Embedding layer
    model.add(Embedding(input_dim=input_dim, output_dim=output_dim, input_length=input_length))

    # Add LSTM
    model.add(LSTM(units=output_dim, return_sequences=True, dropout=0.5, recurrent_dropout=0.5))

    # Add Output Layer
    model.add(Dense(n_tags, activation="softmax"))

    #Optimiser
    # adam = k.optimizers.Adam(lr=0.0005, beta_1=0.9, beta_2=0.999)

    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()

    return model

In [67]:
def train_model(X, y, model):
    loss = list()
    for i in range(25):
        # fit model for one epoch on this sequence
        hist = model.fit(X, y, batch_size=1000, verbose=1, epochs=1, validation_split=0.2)
        loss.append(hist.history['loss'][0])
    return loss

In [68]:
results = pd.DataFrame()
model_lstm = get_lstm_model()
plot_model(model_lstm)
results['with_add_lstm'] = train_model(train_tokens, np.array(train_tags), model_lstm)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 104, 64)           2251456   
                                                                 
 lstm_4 (LSTM)               (None, 104, 64)           33024     
                                                                 
 dense_2 (Dense)             (None, 104, 17)           1105      
                                                                 
Total params: 2285585 (8.72 MB)
Trainable params: 2285585 (8.72 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
26/26 [==============================] - 9s 355ms/step - loss: 0.0705 - accuracy: 0.9796 - val_loss: 0.0707 - val_accuracy: 0.9802


In [69]:
results

,with_add_lstm
0,1.694517
1,0.280205
2,0.250100
3,0.232279
4,0.221959
5,0.212764
6,0.202404
7,0.188429
8,0.168082
9,0.143989


In [22]:
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_sm')
text = nlp('Hi, My name is Rukmini \n I am from India \n I want to work with Google \n Steve Jobs is My Inspiration')
displacy.render(text, style = 'ent', jupyter=True)